In [1]:
import os

In [2]:
%pwd

'f:\\End-to-End-DS-Projects\\Bank Churn Prediction\\research'

In [4]:
# os.chdir("Bank Churn Prediction")
os.chdir("../")

In [5]:
%pwd

'f:\\End-to-End-DS-Projects\\Bank Churn Prediction'

In [6]:
DATA_PATH = 'artifacts/data_ingestion/Churn_Modelling.csv'

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv(DATA_PATH)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [10]:
data.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [11]:
data.shape

(10000, 14)

In [12]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [13]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

#### 4. Update Entity

In [46]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str    
    all_schema: dict
    unzip_dir: Path

#### 5. Update Config manager

In [15]:
from src.BankChurn.constants import *
from src.BankChurn.utils.common import read_yaml, create_directories

In [47]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            unzip_dir=config.unzip_dir 
            
        )

        return data_validation_config

#### 6. Update Components

In [ ]:
# import os
# from src.mlProject import logger

# class DataValiadtion:
#     def __init__(self, config: DataValidationConfig):
#         self.config = config


#     def validate_all_columns(self)-> bool:
#         try:
#             validation_status = None

#             data = pd.read_csv(self.config.unzip_data_dir)
#             all_cols = list(data.columns)

#             all_schema = self.config.all_schema.keys()

            
#             for col in all_cols:
#                 if col not in all_schema:
#                     validation_status = False
#                     with open(self.config.STATUS_FILE, 'w') as f:
#                         f.write(f"Validation status: {validation_status}")
#                 else:
#                     validation_status = True
#                     with open(self.config.STATUS_FILE, 'w') as f:
#                         f.write(f"Validation status: {validation_status}")

#             return validation_status
        
#         except Exception as e:
#             raise e



In [50]:
import os
import pandas as pd
from src.BankChurn import logger
from sklearn.preprocessing import LabelEncoder

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        self.df = pd.read_csv(self.config.unzip_data_dir)
        
    def load_data(self):
        self.df = pd.read_csv(self.config.unzip_data_dir)

    def preprocess(self):
        """ Preprocess the dataframe """      

        # Drop unnecessary columns
        columns_to_drop = ['RowNumber', 'CustomerId', 'Surname']
        for col in columns_to_drop:
            if col in self.df.columns:
                self.df = self.df.drop(columns=[col])

        # Convert columns to integer type
        for col in ['EstimatedSalary', 'Balance']:
            if col in self.df.columns:
                self.df[col] = self.df[col].astype(int, errors='ignore')

        # Encode categorical columns using LabelEncoder
        le = LabelEncoder()
        categorical_cols = ['Geography', 'Gender']
        for col in categorical_cols:
            if col in self.df.columns:
                self.df[col] = le.fit_transform(self.df[col])

        # Drop additional column
        if 'HasCrCard' in self.df.columns:
            self.df = self.df.drop('HasCrCard', axis=1)
            
    
        return self.df
    

    def validate_all_columns(self, preprocessed_data: pd.DataFrame) -> bool:
        try:
            validation_status = True
            all_cols = list(preprocessed_data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    return validation_status
                else:
                    schema_data_type = self.config.all_schema[col]
                    data_data_type = preprocessed_data[col].dtype

                    if schema_data_type != data_data_type:
                        validation_status = False
                        with open(self.config.STATUS_FILE, 'w') as f:
                            f.write(f"Validation status: {validation_status}")
                        return validation_status

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e



#### 7. Update Pipeline

In [70]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    preprocessed_data = data_validation.preprocess()
    validation_status = data_validation.validate_all_columns(preprocessed_data)
    print(f"Validation status: {validation_status}")
except Exception as e:
    logger.error(f"Error occurred: {e}")
    raise e

[2024-06-03 09:23:40,506: 31 - Bank Churn Project Logger: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-03 09:23:40,510: 31 - Bank Churn Project Logger: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-03 09:23:40,519: 31 - Bank Churn Project Logger: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-03 09:23:40,521: 51 - Bank Churn Project Logger: INFO: common: created directory at: artifacts]
[2024-06-03 09:23:40,524: 51 - Bank Churn Project Logger: INFO: common: created directory at: artifacts/data_validation]


Validation status: False


In [53]:
data_validation = DataValiadtion(config=data_validation_config)
preprocessed_data = data_validation.preprocess()
print(preprocessed_data.info())

validation_status

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   CreditScore      10000 non-null  int64
 1   Geography        10000 non-null  int32
 2   Gender           10000 non-null  int32
 3   Age              10000 non-null  int64
 4   Tenure           10000 non-null  int64
 5   Balance          10000 non-null  int32
 6   NumOfProducts    10000 non-null  int64
 7   IsActiveMember   10000 non-null  int64
 8   EstimatedSalary  10000 non-null  int32
 9   Exited           10000 non-null  int64
dtypes: int32(4), int64(6)
memory usage: 625.1 KB
None


False

In [59]:
import dill

EXPORT_PATH = '../artifacts/data_preprocessed/1_preprocessed_df.pkl'

# Export preprocessed_data dataframe to a pickle file
def export_preprocessed_data(df):
    with open(EXPORT_PATH, 'wb') as f:
        dill.dump(df, f)

In [62]:
EXPORT_PATH = '../artifacts/data_preprocessed/1_preprocessed_df.pkl'

preprocessed_data.to_pickle(EXPORT_PATH)

In [64]:
pd.read_pickle('../artifacts/data_preprocessed/1_preprocessed_df.pkl')

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0,1,1,101348,1
1,608,2,0,41,1,83807,1,1,112542,0
2,502,0,0,42,8,159660,3,0,113931,1
3,699,0,0,39,1,0,2,0,93826,0
4,850,2,0,43,2,125510,1,1,79084,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0,2,0,96270,0
9996,516,0,1,35,10,57369,1,1,101699,0
9997,709,0,0,36,7,0,1,1,42085,1
9998,772,1,1,42,3,75075,2,0,92888,1


In [67]:
preprocessed_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0,1,1,101348,1
1,608,2,0,41,1,83807,1,1,112542,0
2,502,0,0,42,8,159660,3,0,113931,1
3,699,0,0,39,1,0,2,0,93826,0
4,850,2,0,43,2,125510,1,1,79084,0
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0,2,0,96270,0
9996,516,0,1,35,10,57369,1,1,101699,0
9997,709,0,0,36,7,0,1,1,42085,1
9998,772,1,1,42,3,75075,2,0,92888,1
